In [1]:
import requests

In [2]:
base_url = "https://api.pushshift.io/reddit/search/submission"

In [17]:
params = {
    "subreddit": "LifeProTips",
    "size": 500
}

In [18]:
res = requests.get(base_url, params)

In [19]:
res.status_code

200

In [20]:
data = res.json()  # Save the JSON

In [21]:
posts = data["data"]  # Save the posts

In [22]:
len(posts)  # getting 25 posts back from the API

500